In [84]:
%load_ext autoreload
%autoreload 2

import util as yu
from util import *
import util_moments as yum

yu.setpath('check_KyriakosStrategy')

projs=['P0', 'Px', 'Py', 'Pz']
inserts=['tt', 'tx', 'ty', 'tz', 'xx', 'xy', 'xz', 'yy', 'yz', 'zz']
enss=['b','c','d']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# avgx

In [8]:
# conn
path='pkl/analysis_avgx/reg_ignore/key2bare_conn.pkl'
key2bare_conn=yu.load_pkl(path)

# for key in key2bare_conn.keys():
#     print(key,yu.jackme_un2str(key2bare_conn[key]))

In [9]:
ens2c2pt={}; ens2moms_2pt={}; ens2c2pt0={}; ens2Njk={}
for ens in enss:
    basepath=f'/p/project1/ngff/li47/code/projectData/05_moments/{yu.ens2full[ens]}/data_merge/'
    path=f'{basepath}disc_2pt.h5'
    with h5py.File(path) as f:
        moms_2pt=yu.moms2list(f['moms'])
        c2pt=yu.jackknife(np.real(f['data/N_N'][:,:,:]))
        
    ens2moms_2pt[ens]=moms_2pt
    ens2c2pt[ens]=c2pt
    ens2c2pt0[ens]=c2pt[:,:,moms_2pt.index([0,0,0])]
    ens2Njk[ens]=len(c2pt)

key2tf2ratio={}
js=['j+;disc','js;disc','jc;disc']+[f'jg;stout{stout}' for stout in [5,7,10,15,20]]
for ens in enss:
    path=f'/p/project1/ngff/li47/code/scratch/run/05_moments_run5/{yu.ens2full[ens]}/data_merge/disc_0,0,1,0,0,0.h5'
    with h5py.File(path) as f:
        for jtf in f['data'].keys():
            j,tf=jtf.split('_'); tf=int(tf)
            if j not in js:
                continue        
            key=(ens,j)
            if key not in key2tf2ratio:
                key2tf2ratio[key]={}
            c3pt=yu.jackknife(f['data'][jtf][:,:,0,projs.index('P0'),inserts.index('tz')])
            c2pt=ens2c2pt[ens][:,tf,ens2moms_2pt[ens].index([0,0,1])]
            factor=1/(1j*2*np.pi/yu.ens2NL[ens])
            ratio=np.real(c3pt/c2pt[:,None]*factor)
            key2tf2ratio[key][tf]=ratio
            
ens2tfs_disc={}
for ens in enss:
    tfs=list(key2tf2ratio[(ens,'j+;disc')].keys()); tfs.sort()
    ens2tfs_disc[ens]=tfs
    print(ens,tfs[0],tfs[-1])
    
path='pkl/analysis_c2pt/reg_ignore/ens2pars_jk_meffnst_selected.pkl'
[ens2pars_jk_meff1st,ens2pars_jk_meff2st,ens2pars_jk_meff3st]=yu.load_pkl(path)

b 2 22
c 2 26
d 2 30


In [13]:
selections=['bandfit_WA']
selection2key2bare={selection:key2bare_conn.copy() for selection in selections}

overwrite=True
def createDic(key):
    ens,j=key
    gett=lambda t:round(t/yu.ens2a[ens])
    def get_tfs(tmin,tmax,dt=1):
        return range(gett(tmin),gett(tmax),dt)
    lat_a=yu.ens2a[ens]
    
    tcmins_1st=[gett(0.2)]
    tfmins_2st=get_tfs(0.5,0.8)
    tcmins_2st=[gett(0.2)]
    
    tf2ratio=yu.cut_tf2ratio(key2tf2ratio[(ens,j)],gett(1.5))
    fits_band=yu.doFits_3pt_band(tf2ratio,tcmins_1st,corrQ=False,label=f'{ens}_{j}_band',overwrite=overwrite)
    fit_band_WA=yu.doWA_band(fits_band,tf_min=gett(0.8),tf_max=gett(1.1),tcmin=max(gett(0.2),2),corrQ=False)
    fits_sum=yu.doFits_3pt('sum',tf2ratio,tfmins_2st,tcmins_2st,label=f'{ens}_{j}_sum',overwrite=overwrite)
    
    selection2key2bare['bandfit_WA'][key]=fit_band_WA[0][:,0]
    
    dic={
        'base:[tf2ratio,fits_band,fits_const,fits_sum,fits_2st]':[tf2ratio,fits_band,None,fits_sum,None],
        'WAMA:[fit_band_WA,fit_const_MA,fit_sum_MA,fit_2st_MA]':[fit_band_WA,None,None,None],
        'rainbow:[tfmin,tfmax,tcmin,dt]':[gett(0.5),gett(1.1),2,1],
        'fit_band:[tfmin,tfmax,tcmin_min,tcmin_max]':[gett(0.5),gett(1.1),gett(0.2),gett(0.2)],
        'fit_sum:[tfmin_min,tfmin_max,tcmin_min,tcmin_max]':[None,None,gett(0.2),gett(0.2)],
        'xunit':yu.ens2a[ens],
    }
    return dic

def createDic2(key):
    ens,j=key
    gett=lambda t:round(t/yu.ens2a[ens])
    def get_tfs(tmin,tmax,dt=1):
        return range(gett(tmin),gett(tmax),dt)
    lat_a=yu.ens2a[ens]
    
    # tfmins_1st=get_tfs(0.8,1.2)
    tcmins_1st=[gett(0.2)]
    tfmins_2st=get_tfs(0.4,1.0)
    tcmins_2st=[gett(0.2)]
    
    tf2ratio=yu.cut_tf2ratio(key2tf2ratio[(ens,j)],gett(1.5))
    fits_band=yu.doFits_3pt_band(tf2ratio,tcmins_1st,corrQ=False,label=f'{ens}_{j}_band',overwrite=overwrite)
    fit_band_WA=yu.doWA_band(fits_band,tf_min=gett(0.95),tf_max=gett(1.5),tcmin=max(gett(0.2),2),corrQ=False)
    fits_sum=yu.doFits_3pt('sum',tf2ratio,tfmins_2st,tcmins_2st,label=f'{ens}_{j}_sum',overwrite=overwrite)
    
    selection2key2bare['bandfit_WA'][key]=fit_band_WA[0][:,0]
    
    dic={
        'base:[tf2ratio,fits_band,fits_const,fits_sum,fits_2st]':[tf2ratio,fits_band,None,fits_sum,None],
        'WAMA:[fit_band_WA,fit_const_MA,fit_sum_MA,fit_2st_MA]':[None,fit_band_WA,None,None],
        'rainbow:[tfmin,tfmax,tcmin,dt]':[gett(0.5),gett(1.5),2,2],
        'fit_band:[tfmin,tfmax,tcmin_min,tcmin_max]':[gett(0.5),gett(1.5),gett(0.2),gett(0.2)],
        'fit_band:[dt]':[2],
        'fit_sum:[tfmin_min,tfmin_max,tcmin_min,tcmin_max]':[None,None,gett(0.2),gett(0.2)],
        'xunit':yu.ens2a[ens],
    }
    return dic

# js_plt=['j+;disc']; enss_plt=['b']
js_plt=js; enss_plt=enss

for j in js_plt:
    keys=[(ens,j) for ens in enss_plt]

    list_dic=[createDic2(key) if key[1].startswith('jg') else createDic(key) for key in keys]

    colors=['b','orange','g','r','purple','brown','magenta','gold','cyan']+['darkblue','darkgreen','darkred','grey']
    yu.makePlot_3pt(list_dic,shows=['rainbow','fit_band','fit_sum'],colors_rainbow=colors,colors_fit=['g'])
    yu.finalizePlot(j)

In [11]:
# RCs

ens2Njk={ens:len(ens2c2pt[ens]) for ens in enss}

path='data_aux/RCs.pkl'
with open(path,'rb') as f:
    ens2RCs_me=pickle.load(f)
ens2RCs={ens:{} for ens in enss}
for ens in enss:
    for key in ens2RCs_me[ens]:
        if key.endswith('err'):
            continue
        ens2RCs[ens][key]=yu.jackknife_pseudo(ens2RCs_me[ens][key],ens2RCs_me[ens][f'{key}_err']+1e-10,ens2Njk[ens])[:,0]
        
path='/p/project1/ngff/li47/code/glwc2/project2/02_discNJN_1D/data_aux/RCs_pre.pkl'
with open(path,'rb') as f:
    ens2RCs_pre_me=pickle.load(f)
ens2RCs_pre={ens:{} for ens in enss[:1]}
for ens in enss[:1]:
    for key in ens2RCs_pre_me[ens]:
        if key.endswith('err'):
            continue
        ens2RCs_pre[ens][key]=yu.jackknife_pseudo(ens2RCs_pre_me[ens][key],ens2RCs_pre_me[ens][f'{key}_err']+1e-10,ens2Njk[ens])[:,0]

In [14]:
selection2key2phy={}
selection2key2phy_pre={}
for selection in selections:
    key2bare=selection2key2bare[selection]
    yum.extendBare_avgx(key2bare)
    key2phy=yum.bareRC2phy_avgx(key2bare,ens2RCs)
    key2phy_pre=yum.bareRC2phy_avgx_pre(key2bare,ens2RCs,ens2RCs_pre)
    
    selection2key2phy[selection]=key2phy
    selection2key2phy_pre[selection]=key2phy_pre

In [15]:
figs=[]
stouts=[5,7,10,15,20]
list_dic=[
    {'key2phy':yum.convert_key2phy_stout(selection2key2phy[selection],stout),
    'key2phy_pre':yum.convert_key2phy_stout(selection2key2phy_pre[selection],stout)}
    for stout in stouts
]
fig,axs=yum.makePlot_a2dependence_avgx(list_dic)
yu.addColHeader(axs,[f'stout{stout}' for stout in stouts])
yu.finalizePlot(closeQ=True)
figs.append(fig)
    
yu.makePDF('a2dep',figs)

# B20

Kyriakos didn't use the error to do SVD

In [91]:
msqs=range(25)
Q2s=np.array([yum.n2qpp12Q2((msq,msq,0),'b') for msq in msqs])
def Q2_to_msq(Q2):
    ind=np.argmin(np.abs(Q2-Q2s))
    msq=msqs[ind]
    # print(Q2,Q2s[ind],msq)
    return msq

path='data_aux/dat_ignore/Kyriakos_isoscalar/'
key2me_Kyriakos={}
for file in os.listdir(path):
    ff,tf=file[:-4].split('_'); tf=int(tf[2:])
    with open(f'{path}{file}','r') as f:
        t=f.readlines()
    for ele in t:
        Q2,tc,m,e=ele[:-2].split(' ')
        key=(ff,'j+;conn',Q2_to_msq(float(Q2)),tf,int(tc))
        key2me_Kyriakos[key]=(float(m),float(e))
        
path='data_aux/dat_ignore/B20_CK/'
key2dat_CK={}
for file in os.listdir(path):
    msq=int(file[-5:-3])
    with h5py.File(f'{path}{file}') as f:
        msqkey=list(f.keys())[0]
        for tfkey in f[msqkey].keys():
            tf=int(tfkey[2:])
            for iso in ['isov','isos']:
                j={'isos':'j+;conn','isov':'j-;conn_R'}[iso]
                for ff in ['A20','B20','C20']:
                    key=(ff,j,msq,tf)
                    key2dat_CK[key]=f[msqkey][tfkey][iso][ff]['data'][:]

In [129]:
ens2RCs_me=yu.load_pkl('data_aux/RCs.pkl')
def run(ff,j,ens):
    figs=[]
    c1='Kyriakos'; c2='0err'
    n2qpp1s=[(6,6,0),(12,12,0)]
    # n2qpp1s=[(1,1,0),(2,2,0),(3,3,0),(4,4,0),(6,6,0),(8,8,0),(9,9,0),(10,10,0),(11,11,0),(12,12,0),(13,13,0),(14,14,0),(16,16,0)]
    for n2qpp1 in n2qpp1s:
        n2q,n2p,n2p1=n2qpp1
        case2tf2ratio={}
        path=f'/p/project1/ngff/li47/code/projectData/05_moments/doSVD_Kyriakos/conn_{ens}_{yu.n2qpp12str(n2qpp1)}.h5'
        with h5py.File(path) as f:
            for case in f.keys():
                if case not in case2tf2ratio.keys():
                    case2tf2ratio[case]={}
                for key in f[case].keys():
                    tff,tj,tf=key.split('_'); tf=int(tf)
                    if (tff,tj) != (ff,j):
                        continue
                    case2tf2ratio[case][tf]=f[case][key][:]

        cases_do=[f'{c1}_{c2}']
        
        def createDic(ind,case):
            gett=lambda t:round(t/yu.ens2a[ens])
            def get_tfs(tmin,tmax,dt=1):
                return range(gett(tmin),gett(tmax),dt)
        
            tfmins_1st=[8,10,12,14,16,18,20]
            tcmins_1st=yu.cutExtraDiff2tcmins([0,1,2,3,4,5,6,7],[-3,-2,-1,0,1,2,3],cutBase=2)
            
            tf2ratio=case2tf2ratio[case]
                    
            dic={
                'base:[tf2ratio,fits_band,fits_const,fits_sum,fits_2st]':[tf2ratio,None,None,None,None],
                'rainbow:[tfmin,tfmax,tcmin,dt]':[None,None,2,None],
                'xunit':yu.ens2a[ens],
                # 'yunit':yunits[ind],
            }
            return dic
        list_dic=[createDic(ind,case) for ind,case in enumerate(cases_do)]

        colors=['b','orange','g','r','purple','brown','magenta','gold','cyan']+['darkblue','darkgreen','darkred','grey']
        fig,axs=yu.makePlot_3pt(list_dic,shows=['rainbow','midpoint'],sharey=True,colors_rainbow=colors)
        xunit=yu.ens2a[ens]; yunit=1
        
        tfs_rainbow=tfs=[8,10,12,14,16,18,20]
        
        def plt(tf2ratio_me,shift=0):
            for tf in tfs:
                mean,err=tf2ratio_me[tf]
                
                itf=tfs.index(tf)
                tcmin_rainbow=2
                tcs=np.arange(tcmin_rainbow,tf-tcmin_rainbow+1)
                itf_color=tfs_rainbow.index(tf)
                
                ax=axs[0,0]
                plt_x=(tcs-tf/2+0.05*(itf-len(tfs_rainbow)/2)+shift)*xunit; plt_y=mean[tcs]*yunit; plt_yerr=err[tcs]*yunit
                ax.errorbar(plt_x,plt_y,plt_yerr,color=colors[itf_color%16],fmt=yu.fmts16[itf_color%16],alpha=0.3)
                
                ax=axs[0,1]
                plt_x=(tf+shift*2)*xunit; plt_y=mean[tf//2]*yunit; plt_yerr=err[tf//2]*yunit
                ax.errorbar(plt_x,plt_y,plt_yerr,color=colors[itf_color%16],fmt=yu.fmts16[itf_color%16],alpha=0.3)

        try:
            tf2ratio_me={}
            for tf in tfs:
                mean=np.array([key2me_Kyriakos[('B20',j,n2q,tf,tc)][0] for tc in range(tf+1)])
                err=np.array([key2me_Kyriakos[('B20',j,n2q,tf,tc)][1] for tc in range(tf+1)])
                tf2ratio_me[tf]=(mean,err)
            tf2ratio_me_Kyriakos=tf2ratio_me
            plt(tf2ratio_me_Kyriakos,-0.1)
        except:
            pass
        
        # try:
        #     tf2ratio_me={}
        #     for tf in tfs:
        #         t=key2dat_CK[('B20',j,n2q,tf)]
        #         mean,err=yu.jackme(t)
        #         tf2ratio_me[tf]=(mean,err)
        #     tf2ratio_me_CK=tf2ratio_me
        #     plt(tf2ratio_me_CK,0.1)
        # except:
        #     pass
        
        
        for i in range(len(axs)):
            axs[i,0].set_ylabel(rf'$Q^2$={yum.n2qpp12Q2(n2qpp1,ens):.4f}')
        if n2qpp1==(6,6,0):
            axs[0,0].set_ylim([-0.1,0.1])
            axs[0,0].set_yticks([-0.1,-0.05,0,0.05,0.1])
        if n2qpp1==(12,12,0):
            axs[0,0].set_ylim([-0.1,0.15])
            axs[0,0].set_yticks([-0.1,-0.05,0,0.05,0.1,0.15])
        yu.finalizePlot(closeQ=True)
        figs.append(fig)

    yu.makePDF(f'B20_rainbow',figs)

ff='B20'; j='j+;conn'; ens='b'
run(ff,j,ens)